In [1]:
#  SBERT 를 위한 패키지인 sentence_transformers, 형태소 분석기 KoNLPy 설치
!pip install sentence_transformers
!pip install konlpy

import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 727 kB/s 
     |████████████████████████████████| 5.8 MB 85.5 MB/s 
     |████████████████████████████████| 1.3 MB 89.1 MB/s 
     |████████████████████████████████| 182 kB 83.4 MB/s 
     |████████████████████████████████| 7.6 MB 72.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9786dc9fe7999c38c47f139bbca136581c759e9ff25e97175680105cd7d2755e
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 982 kB/s 
     |████████████████████████████████| 465 kB 89.1 MB/s 


In [2]:
doc = """
나와 인태가 고기가 먹고 싶다고 하여 육장장이로 향했는데, 연길이는 그다지 내켜하지 않았습니다. 이 부분이 조금 더 나를 즐겁게 했습니다. 하하. 
육장장이는 고기 무한리필집 중에서 조금 비싼 편에 속해서인지 갈 때마다 고기 차이가 많이 나지 않기에 고기는 맛있게 먹었습니다. 
그러나, 사이드 메뉴가 조금 맛이 이상해서 안타까웠습니다. 
저녁 식사 이후, ‘밥’ 모임끼리 겨울 여행 계획을 짜면서 재미있는 하루를 보냈습니다. 
"""

# 형태소 분석기를 통해 명사만 추출한 문서 만들기
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

# 사이킷런의 CountVectorizer를 사용해 단어 추출 / 이유: n_gram_range의 인자를 사용하면 쉽게 n-gram 추출이 가능하기 때문
n_gram_range = (1, 2) # 결과 후보는 2 개의 단어를 한 묶음으로 간주하는 bigram 과 3 개의 단어를 한 묶음으로 간주하는 trigram 을 추출

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

# 한국어를 포함하고 있는 다국어 SBERT 를 로드
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc]) # 문서 encoding하기
candidate_embeddings = model.encode(candidates) # 추출된 단어 인코딩하기

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [3]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
  # 문서와 각 키워드들 간의 유사도
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  
  # 각 키워드들 간의 유사도
  distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)
  
  # 코사인유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
  words_idx = list(distances.argsort()[0][-nr_candidates:])
  words_vals = [candidates[index] for index in words_idx]
  distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]
  
  # 각 키워드들중에서 가장 덜 유사한 키워드들 간의 조합을 계산 (# 상위 10 개의 키워드를 선택하고 이 10 개 중에서 서로 가장 유사성이 낮은 5 개를 선택)
  min_sim = np.inf
  
  candidate = None
  for combination in itertools.combinations(range(len(words_idx)), top_n):
    sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
    if sim < min_sim:
      candidate = combination
      min_sim = sim
      
  return [words_vals[idx] for idx in candidate]

In [4]:
print(max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30))

['차이', '모임 끼리', '겨울 여행', '고기 무한리필', '육장 고기']


In [5]:
# 카테고리 encoding
category= '두려움, 사랑, 성찰, 인생, 타인, 책임, 자신, 능력, 휴식'
tokenized_category = okt.phrases(category)
category_count = CountVectorizer(ngram_range=(1,1)).fit(tokenized_category)
category_candidates = category_count.get_feature_names_out()
category_embeddings = model.encode(category_candidates)

In [6]:
sim_keyword = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)
tokenized_sim_keyword = ' '.join([word[:] for word in sim_keyword])
sim_keyword_count = CountVectorizer(ngram_range=(1,2)).fit(sim_keyword)
sim_keyword_candidates = sim_keyword_count.get_feature_names_out()
sim_keyword_embeddings = model.encode(sim_keyword_candidates)
distances_category = cosine_similarity(sim_keyword_embeddings, category_embeddings)

print(tokenized_sim_keyword)

차이 모임 끼리 겨울 여행 고기 무한리필 육장 고기


In [7]:
def total_category(sim_keyword_embeddings, category_embeddings, top_p, nr_candidates):
  # category와 글 키워드로 나온 단어들 간의 코사인 유사도
  distances_category = cosine_similarity(sim_keyword_embeddings, category_embeddings)

  # 코사인유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
  category_idx = list(distances_category.argsort()[0][-nr_candidates:])
  category_vals = [category_candidates[index] for index in category_idx]
  distances_category = distances_category[np.ix_(category_idx, category_idx)]

  min_sim_total = np.inf
  
  total_candidate = None
  for total_combination in itertools.combinations(range(len(category_idx)), top_p):
    sim_total = sum([distances_category[i][j] for i in total_combination for j in total_combination if i != j])
    if sim_total < min_sim_total:
      total_candidate = total_combination
      min_sim_total = sim_total
      
  return [category_vals[idx] for idx in total_candidate]

print(total_category(sim_keyword_embeddings, category_embeddings, top_p=1, nr_candidates=30))

['사랑']
